In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import TrainingArguments
from transformers import AutoModelForSequenceClassification,Trainer
raw_datasets=load_dataset("glue","sst2")
tokenizer=AutoTokenizer.from_pretrained("bert-base-uncased")

def preprocess_function(examples):
  return tokenizer(examples['sentence'],truncation=True,padding="max_length")


tokenized_datasets=raw_datasets.map(preprocess_function,batched=True)

train_subset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
eval_subset = tokenized_datasets["validation"].shuffle(seed=42).select(range(100))


training_arg=TrainingArguments(
    output_dir="./bert-sentiment",
    learning_rate=0.00002,
    per_device_train_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_steps=10
)




model=model=AutoModelForSequenceClassification.from_pretrained("bert-base-uncased",num_labels=2)


trainer=Trainer(
    model=model,
    args=training_arg,
    train_dataset=train_subset,
    eval_dataset=eval_subset
)


trainer.train()






In [ ]:

save_directory = "./my_fine_tuned_bert"
trainer.save_model(save_directory)


tokenizer.save_pretrained(save_directory)

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification


load_directory = "./my_fine_tuned_bert"


loaded_tokenizer = AutoTokenizer.from_pretrained("./my_fine_tuned_bert")


loaded_model = AutoModelForSequenceClassification.from_pretrained("./my_fine_tuned_bert")





print("Model loaded successfully")

In [6]:
import torch

def predict_sentiment(text):

    inputs = loaded_tokenizer(text, return_tensors="pt")


    with torch.no_grad():
        outputs = loaded_model(**inputs)


    logits = outputs.logits


    predicted_class_id = torch.argmax(logits, dim=1).item()


    labels = ["Negative", "Positive"]
    return labels[predicted_class_id]


print(predict_sentiment("I love to learn about new things"))
print(predict_sentiment("he is not satisfied"))

Positive
Negative
